# Optimization for roboticists
### Author: Shivendra Agrawal

Let's try to make sense of some popular terms in Optimization ​
1. Gradient Descent​
2. Steepest Descent​
3. Newton's Method​
4. Quasi-Newton Methods (like BFGS)​
5. Line Search ​
6. Adam​
7. RMSprop​
8. Trust Region
9. Interior Point Methods​ (Linear/Quadratic Programming)​
10. Simplex Method​
11. Branch and Bound & Branch and Cut​
12. Barrier Function​
13. Non-convex optimization​
14. Constrained optimization​
15. Gradient free optimization​
16. Backpropagation

In [ ]:
%pip install ipywidgets
%pip install matplotlib

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, IntSlider, fixed, Output, IntText, Button, VBox, FloatSlider
from IPython.display import display

# Unconstrained non-convex function
def f(x):
    """Objective function."""
    return x**4 - 3*x**3 + 2

### Convex or non-convex
We can find this for a twice differentiable scalar functions using second-derivative known as Hessian.

In [89]:
def hessian(x):
    """Second derivative (Hessian) of the objective function."""
    return 12*x**2 - 18*x

def plot_function(f, second_derivative, x_max=3):
    # Generate x values
    x = np.linspace(0, x_max, 400)

    # Compute f(x) and its second derivative for each x
    y = f(x)
    y_second_derivative = second_derivative(x)

    fig, axs = plt.subplots(2, figsize=(8, 6))

    # Plot f(x)
    axs[0].plot(x, y, label='f(x) = $x^4 - 3x^3 + 2$')
    axs[0].set_title('Function f(x)')
    axs[0].legend()

    # Plot the second derivative of f(x)
    axs[1].plot(x, y_second_derivative, label="f''(x) = $12x^2 - 18x$", color='orange')
    axs[1].axhline(0, color='red', linestyle='--')  # This line indicates where the second derivative is zero
    axs[1].set_title("Second Derivative of f(x)")
    axs[1].legend()

    plt.tight_layout()
    plt.show()

# Create an interactive widget for x_max
interactive(plot_function, f=fixed(f), second_derivative=fixed(hessian), x_max=(1, 5, 0.1))

interactive(children=(FloatSlider(value=3.0, description='x_max', max=5.0, min=1.0), Output()), _dom_classes=(…

#### Visualization of Function and Its Second Derivative

In the plots above, we visualize a function \( f(x) = x^4 - 3x^3 + 2 \) and its second derivative.

##### Function \( f(x) \)

The first plot showcases the function \( f(x) \) against the variable \( x \). The function exhibits a shape defined by the equation, where we can identify the local minima and maxima visually.

##### Second Derivative of \( f(x) \)

The second plot showcases the second derivative of \( f(x) \), denoted as \( f''(x) \). The second derivative informs us about the curvature of the function:

- **Positive values** of the second derivative indicate regions where the function is **convex**. In other words, the function curves upwards.
  
- **Negative values** indicate **concave** regions, where the function curves downwards.

- **Zero values** signify potential **inflection points**, where the curvature changes from convex to concave or vice versa.

By analyzing the second derivative plot, we can discern where the function is convex or concave. The red dashed line marks the point where \( f''(x) = 0 \), which can help in identifying the inflection points.


### Gradient Descent

In [91]:
def gradient(x):
    """First derivative of the objective function."""
    return 4*x**3 - 9*x**2

def gradient_descent(lr=0.01, epochs=10, start_x=1.5):
    """Perform gradient descent optimization.
    
    Args:
    - lr (float): Learning rate.
    - epochs (int): Number of iterations.
    - start_x (float): Starting value of x.
    
    Returns:
    - list: History of x values during optimization.
    - list: History of function values during optimization.
    """
    
    # Initialize x with the starting value
    x = start_x
    
    # Lists to store the history of x values and corresponding function values.
    x_history = []
    f_history = []
    
    # Gradient descent loop.
    for _ in range(epochs):
        # Update rule: Move in the opposite direction of the gradient.
        # Notice that the learning rate is a constant in this case.
        x = x - lr * gradient(x)
        
        # Store the updated values in the history.
        x_history.append(x)
        f_history.append(f(x))
        
    return x_history, f_history

def interactive_gradient_descent(epochs=10):
    """Visualize gradient descent convergence interactively."""
    x_history, f_history = gradient_descent(epochs=epochs)
    x = np.linspace(-1, 3, 400)
    y = f(x)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'r', label="f(x)")
    plt.scatter(x_history, f_history, c='blue', s=50, alpha=0.6, label=f'Gradient Descent (Latest f(x)={f_history[-1]:.2f})')
    
    plt.title('Gradient Descent Convergence')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

interact(interactive_gradient_descent, epochs=IntSlider(min=1, max=20, value=1, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=20, min=1), Output…

### Steepest Descent 
If your function outputs a vector instead of scalar. ​
For example – Minimizing the magnitude of a vector. Each component might also have joint constraints and/or interactions. 

### Newton's method 

More generally Newton-Raphson method is an iterative method for finding the roots of a differentiable function $f$, which are solutions of the equation $f(x)=0$.
In optimization, we try to find the solutions for the first-derivative or zeros of the first-derivative.

The update rule is 

$x = x - gradient(x)/hessian(x)$
* It uses not just the slope (the first derivative) but also the curvature (the second derivative or Hessian) to decide which direction to move in and how far to move.
* The magnitude of the curvature can give an idea about how "steep" or "flat" the valley or peak is. When you divide the gradient by the Hessian in Newton's method, you're effectively scaling your step size by how curved the function is at your current point. If the function is very curved, you take smaller steps; if it's flatter, you take larger steps.

**Fun-fact:** If you do the quadratic approximation of the function at the current best solution and find its zeros, you will get the same update rule so it is as if you are doing quadratic approximation in each iteration and solving for it.
I put this for visualization sake. Actual method doesn't do this approximation just that the update rule is akin to solving for iterative quadratic approximation. 

In [92]:
# Quadratic approximation for Newton's method
# (this is only needed for the visualization)
def quadratic_approximation(x, x_k):
    """Compute the quadratic approximation at a given point x_k."""
    return f(x_k) + gradient(x_k) * (x - x_k) + 0.5 * hessian(x_k) * (x - x_k)**2

def newtons_method(epochs=10, start_x=1.5, epsilon=1e-8):
    """Perform optimization using Newton's method."""
    x = start_x
    x_history = [x]
    f_history = [f(x)]
    
    for _ in range(epochs):
        if abs(hessian(x)) < epsilon:  # Check if Hessian is too close to zero to avoid divide by zero error
            print("Hessian too close to zero. Stopping optimization.")
            break
            
        x = x - gradient(x) / hessian(x)  # This inversion of hessian if it a giant matrix could be expensive
        x_history.append(x)
        f_history.append(f(x))
        
    return x_history, f_history

def interactive_newtons_method(epochs=10, start_x=1.5):
    """Visualize Newton's method convergence interactively."""
    x_history, f_history = newtons_method(epochs=epochs, start_x=start_x)
    x = np.linspace(-2, 4, 400)
    y = f(x)
    
    plt.figure(figsize=(10, 6))
    # Restrict y axis to make the plot more readable
    plt.ylim(-10, 40)
    
    # Plot the original function
    plt.plot(x, y, 'r', label="f(x)")
    
    # Plot the points computed by Newton's method
    plt.scatter(x_history, f_history, c='green', s=50, alpha=0.6, 
                label=f"Newton's Method (Latest f(x)={f_history[-1]:.2f})")
    
    # Plot the quadratic approximation at the latest point
    y_approx = quadratic_approximation(x, x_history[-1])
    plt.plot(x, y_approx, 'b--', label="Quadratic Approximation")
    
    plt.title("Newton's Method Convergence")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

# Invoke the interactive widget for Jupyter Notebook.
interact(interactive_newtons_method, 
         epochs=IntSlider(min=1, max=15, value=1, continuous_update=False),
         start_x=FloatSlider(min=-2, max=4, value=1.6, step=0.2, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=15, min=1), FloatS…

### Quasi-Newton method (BFGS)

Approximates the hessian rather than calculating it and thus speeds up the calculation.

## Newton-like Update for Multi-dimensional Optimization

Given a function $f(\mathbf{x})$ where $\mathbf{x}$ is a vector, the gradient $\nabla f$ gives the direction of steepest ascent, and the Hessian $H$ provides information about the curvature.

The Newton update step in multi-dimensional form is:

$$
\begin{align*}
\mathbf{x}_{\text{new}} &= \mathbf{x}_{\text{old}} - H^{-1} \nabla f(\mathbf{x}_{\text{old}})
\end{align*}
$$

Where:
- $\mathbf{x}_{\text{new}}$ is the updated parameter vector.
- $\mathbf{x}_{\text{old}}$ is the previous parameter vector.
- $H^{-1}$ is the inverse of the Hessian matrix.
- $\nabla f(\mathbf{x}_{\text{old}})$ is the gradient of the function evaluated at $\mathbf{x}_{\text{old}}$.

This update rule essentially adjusts the parameters $\mathbf{x}$ by moving along the direction of the negative gradient, scaled by the inverse of the Hessian.



In [93]:
def bfgs(epochs=10, start_x=1.5, epsilon=1e-8, hessian_inv_init=None):
    """
    Perform optimization using the BFGS quasi-Newton method.
    
    Parameters:
    - epochs: Maximum number of iterations.
    - start_x: Starting point.
    - epsilon: Small value; if gradient magnitude is below this, stop.
    - hessian_inv_init: Initial approximation of the inverse Hessian as a 2D matrix.
    
    Returns:
    - x_history: History of x values during optimization.
    - f_history: History of function values during optimization.
    """
    # If no initial Hessian inverse approximation is given, initialize with identity.
    if hessian_inv_init is None:
        hessian_inv_init = np.eye(1)
    hessian_inv_approximation = hessian_inv_init.copy()
    
    x = start_x
    x_history = []
    f_history = []
    
    for _ in range(epochs):
        grad = gradient(x)
        if np.linalg.norm(grad) < epsilon:
            break
        
        p = -np.dot(hessian_inv_approximation, grad)
        alpha = 0.01
        new_x = x + alpha * p # Akin to Newton's method update rule.
        
        s = np.array(new_x - x).reshape(-1, 1) # Difference between new and old x
        y = (gradient(new_x) - grad).reshape(-1, 1) # Difference between new and old gradient
        
        rho = 1.0 / np.dot(y.T, s)
        I = np.eye(hessian_inv_approximation.shape[0])
        hessian_inv_approximation = (I - rho * np.dot(s, y.T)) @ hessian_inv_approximation @ (I - rho * np.dot(y, s.T)) + rho * np.dot(s, s.T)
        
        x = float(new_x)  # Ensure x remains a scalar
        
        # Store the current point and its function value for visualization at the end of iteration.
        x_history.append(x)
        f_history.append(f(x))
    
    return x_history, f_history

def interactive_bfgs(epochs=10, start_x=1.5):
    x_history, f_history = bfgs(epochs=epochs, start_x=start_x)
    x = np.linspace(-2, 4, 400)
    y = f(x)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'r', label="f(x)")
    plt.scatter(x_history, f_history, c='blue', s=50, alpha=0.6, label=f"BFGS Steps (Latest f(x)={f_history[-1]:.2f})")
    plt.title("BFGS Method Convergence")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

interact(interactive_bfgs, 
         epochs=IntSlider(min=1, max=400, value=1, continuous_update=False),
         start_x=FloatSlider(min=-2, max=4, value=1.6, step=0.2, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=400, min=1), Float…

### Line Search

So far we found the 'direction' of update but kept the learning rate $\alpha$ fixed.
The goal of the line search is to adaptively choose the step size $\alpha$ to ensure a sufficient decrease in the function value at each step.

**Procedure**:

- We start with a tentative step size, $\alpha$.
- We check if moving from $x$ in the given direction by $\alpha$ leads to a "sufficient" decrease in the function value. 
- The notion of "sufficient" is determined by the Armijo-Goldstein condition:

$$
f(x + \alpha \times \text{direction}) > f(x) + c \times \alpha \times \nabla f(x)^T \times \text{direction}
$$

The right side of this inequality is a linear approximation of the function around the point $x$, increased by a fraction $c$ of the expected reduction. If the actual function value on moving by $\alpha$ in the given direction is more than this, it means the step size is too large and might be overshooting the minimum.
- If the above condition is satisfied, we reduce $\alpha$ by multiplying it with $\rho$ and check again.
- This process continues iteratively until we find an $\alpha$ that satisfies the condition, ensuring a sufficient decrease.

In [94]:
def backtracking_line_search(x, direction, alpha=1.0, rho=0.5, c=0.5):
    """
    Perform backtracking line search.
    
    x: Current position
    direction: Descent direction
    alpha: Starting step size (default 1.0)
    rho: Factor to decrease alpha if insufficient decrease (default 0.5)
    c: Fraction of decrease we'd like (default 0.5)
    
    Returns:
    - Optimal step size alpha.
    """
    while f(x + alpha * direction) > f(x) + c * alpha * gradient(x) * direction:
        alpha = rho * alpha
    return alpha

def gradient_descent_with_line_search(epochs=10, start_x=1.5):
    x = start_x
    x_history = []
    f_history = []
    alpha_history = []
    
    for _ in range(epochs):
        grad = gradient(x)
        
        # Compute the direction of descent.
        p = -grad
        
        # Use backtracking line search to compute step size.
        alpha = backtracking_line_search(x, p)
        alpha_history.append(alpha)
        
        # Update the current point in the direction of descent.
        x = x + alpha * p
        
        # Store the current point and its function value for visualization.
        x_history.append(x)
        f_history.append(f(x))
    
    return x_history, f_history, alpha_history

def interactive_gd_line_search(epochs=10, start_x=1.5):
    x_history, f_history, alpha_history = gradient_descent_with_line_search(epochs=epochs, start_x=start_x)
    x = np.linspace(-2, 4, 400)
    y = f(x)
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))
    
    axs[0].plot(x, y, 'r', label="f(x)")
    axs[0].scatter(x_history, f_history, c='blue', s=50, alpha=0.6, label=f"Gradient Descent Steps (Latest f(x)={f_history[-1]:.2f})")
    axs[0].set_title("Gradient Descent with Line Search Convergence")
    axs[0].set_xlabel('x')
    axs[0].set_ylabel('f(x)')
    axs[0].legend(loc='upper left')
    axs[0].grid(True)
    
    axs[1].plot(range(len(alpha_history)), alpha_history, '-o', color="green", label="Alpha (Step Size)")
    axs[1].set_title("Step Size (Alpha) at each Iteration")
    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Alpha')
    axs[1].legend(loc='upper right')
    axs[1].grid(True)
    
    plt.tight_layout()
    plt.show()

interact(interactive_gd_line_search, 
         epochs=IntSlider(min=1, max=30, value=1, continuous_update=False),
         start_x=FloatSlider(min=-2, max=4, value=-1.0, step=0.2, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=30, min=1), FloatS…

### RMSprop

* Way be cleverly find the learning rate specifically for deep learning.
* It uses the running average of the squares of the gradients.
* Maintains a per-parameter learning rate, adjusted by the magnitude of the recent gradients. Parameters associated with frequent high gradients get a reduced learning rate, and those with small gradients get an increased learning rate.

- Also helps with vanishing and exploding gradients
- Better for nonstationary data by focuing on recent gradients than distant past gradients.




In [95]:
def rmsprop(lr=0.1, gamma=0.9, epsilon=1e-8, epochs=100, start_x=1.5): # NOtice a larger learning rate
    x = start_x
    moving_avg_sq = 0
    x_history = []
    f_history = []
    
    for _ in range(epochs):
        g = gradient(x)
        moving_avg_sq = gamma * moving_avg_sq + (1 - gamma) * g**2
        x = x - lr / (np.sqrt(moving_avg_sq) + epsilon) * g
        
        x_history.append(x)
        f_history.append(f(x))
    
    return x_history, f_history

# Visualization
def interactive_rmsprop(epochs=10, start_x=1.5):
    x_history, f_history = rmsprop(epochs=epochs, start_x=start_x)
    x = np.linspace(-2, 4, 400)
    y = f(x)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'r', label="f(x)")
    plt.scatter(x_history, f_history, c='blue', s=50, alpha=0.6, label=f"RMSprop Steps (Latest f(x)={f_history[-1]:.2f})")
    plt.title("RMSprop Convergence")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

interact(interactive_rmsprop, 
        epochs=IntSlider(min=1, max=1000, value=1, continuous_update=False),
        start_x=FloatSlider(min=-2, max=4, value=-1, step=0.2, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=1000, min=1), Floa…

### Adam

Combines the ideas of 
1. RMSprop - By accounting for exponentially decaying average of past squared gradients.
2. Momentum - By accounting for an exponentially decaying average of past gradients to retain a small portion of changes from previous changes. 

In [96]:
def adam(lr=0.1, beta1=0.9, beta2=0.999, epsilon=1e-8, epochs=100, start_x=-1):
    x = start_x
    m = 0  # Moving average of the gradients (momentum).
    v = 0  # Moving average of the squared gradients.
    t = 0  # Time step.
    
    x_history = []
    f_history = []
    
    for _ in range(epochs):
        t += 1
        grad = gradient(x)
        
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad**2
        
        m_corr = m / (1 - beta1**t)  # Bias-corrected estimate of the first moment (mean).
        v_corr = v / (1 - beta2**t)  # Bias-corrected estimate of the second moment (variance).
        
        x -= lr * m_corr / (np.sqrt(v_corr) + epsilon)
        
        x_history.append(x)
        f_history.append(f(x))
    
    return x_history, f_history

def interactive_RMSprop_adam(epochs=10, start_x=-1.5):
    x_rmsprop, f_rmsprop = rmsprop(epochs=epochs, start_x=start_x)
    x_adam, f_adam = adam(epochs=epochs, start_x=start_x)
    
    x = np.linspace(-2, 4, 400)
    y = f(x)
    
    plt.figure(figsize=(14, 8))
    plt.plot(x, y, 'r', label="f(x)")
    
    plt.scatter(x_rmsprop, f_rmsprop, c='blue', s=50, alpha=0.6, label=f"RMSprop (Latest f(x)={f_rmsprop[-1]:.2f})")
    plt.scatter(x_adam, f_adam, c='green', marker='x', s=50, alpha=0.6, label=f"Adam (Latest f(x)={f_adam[-1]:.2f})")
    
    plt.title("RMSprop vs. Adam Convergence")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

interact(interactive_RMSprop_adam, 
         epochs=IntSlider(min=1, max=200, value=1, continuous_update=False),
         start_x=FloatSlider(min=-2, max=4, value=-1.5, step=0.2, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='epochs', max=200, min=1), Float…

RMSprop can still do better in some scenarios because of more stability, less memory overhead. For example RMSprop has shown better results in some cases in Deep RL.  

### Trust Region Methods

* They are iterative optimization methods that constrain each iteration to lie within a specific region where the optimization model (often a quadratic approximation) is believed to be trustworthy. They can still use gradient descent. They are used instead of line search.

**Gradient Descent / Line Search:**
- Choose a direction.
- Decide how far to step in that direction.

**Trust Region:**
- Define a region around the current point.
- Choose a step and direction simultaneously such that the new point lies within this region.
- Adjust the size of the region based on how trustworthy the approximation was.

**When to use them**

In [ ]:
'''
Initialize x, Delta (size of trust region)
while not converged:
    Form a model Q of the function around x (often a quadratic approximation)
    Solve the trust region subproblem: 
        p = argmin(Q) subject to ||p|| <= Delta
    Evaluate the ratio of actual reduction in f to predicted reduction 
    if the ratio is high:
        x = x + p
    Adjust Delta based on the ratio (increase if high, decrease if low)
'''

# Let's do it for a simple quadratic function f(x) = x^2
def f(x):
    return x**2

def df(x):
    return 2*x # Decrease predicted by the quadratic model

def trust_region(epochs=10, initial_delta=1.0, eta=0.2):
    x = 1.5  # Starting from x = 1.5 for visualization purposes
    x_vals = [x]
    f_vals = [f(x)]
    delta_vals = [initial_delta]
    delta = initial_delta
    
    for _ in range(epochs):
        gradient = df(x)
        
        # If the gradient is very close to zero, break out
        if np.abs(gradient) < 1e-8:
            break
        
        model_decrease = 0.5 * gradient**2
        
        x_new = x - (delta / np.abs(gradient)) * gradient
        actual_decrease = f(x) - f(x_new)
        
        # Protect against division by zero
        if model_decrease == 0:
            ratio = 0
        else:
            ratio = actual_decrease / model_decrease
        
        # Adjust the trust region size based on the ratio
        if ratio < 0.9:
            delta *= 0.5
        elif ratio > 0.99:
            delta = min(2.0 * delta, 2*initial_delta)  # capping the maximum size
        # if ratio is between 0.9 and 0.99, delta remains unchanged
        
        x = x_new
        x_vals.append(x)
        f_vals.append(f(x))
        delta_vals.append(delta)
    
    return x_vals, f_vals, delta_vals

def interactive_trust_region(epochs=1):
    x_vals, f_vals, delta_vals = trust_region(epochs=10)  # We always compute the whole sequence for simplicity
    x_range = np.linspace(-1, 3, 400)
    y_range = f(x_range)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x_range, y_range, 'r', label="f(x)")
    plt.scatter(x_vals[:epochs+1], f_vals[:epochs+1], c='blue', s=50, alpha=0.6, label="Trust Region Steps")
    
    # Plot the trust region boundary
    for i in range(epochs):
        circle = plt.Circle((x_vals[i], f_vals[i]), delta_vals[i], color='b', fill=False, linestyle='--', linewidth=1)
        plt.gca().add_artist(circle)
    
    plt.title(f"Trust Region Optimization: Epoch {epochs}")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.grid(True)
    plt.ylim(-0.5, 3)  # Fixing y-range for better visualization
    plt.show()

interact(interactive_trust_region, epochs=IntSlider(min=1, max=3, value=1, continuous_update=False));

### Least-squares

https://www.cvxpy.org/examples/basic/least_squares.html

This can be solved analyticall for a small number of data but we will have to rely on convex optimization methods to otherwise solve it. 

### Stochastic Gradient Descent

- Uses just one example to change weights instead of a 'batch' change. 
- Tends to be very oscillating. 
- It is usuallyy better to use mini-batch Gradient Descent

### Backpropagation

- It's an algorithm to calculate the gradient of the loss function with respect to each weight in the neural network.
- In essence, backpropagation computes how much each weight contributes to the error.

### Lagrangian Multipliers

- Used primarily for equality constraints. 
- The basic idea is to transform a constrained problem into an unconstrained one by introducing multipliers for each constraint and forming a "Lagrangian" function.

### Interior Point Methods
 - Used to solve **Linear Programming** and **Quadratic Programming**.
 - At the core uses Newton's Method combined with various strategies such as Barrier Functions for constrained optimization.
 - Popular open source solvers like  ECOS (Efficient Cone Solver) use this along with many strategies. 

 **Barrier Functions**
 A barrier function is introduced into the objective function to ensure that the iterative methods stay within the feasible region.

### Gradient-free Optimization

Usually means estimating the gradient

**STOMP**
- STOMP is a gradient-free optimization method.
- Instead of computing gradients, uses noisy rollouts to estimate the gradient information.


### Simplex Method

Used only for Linear Programming where the objective and constraints are linear. 
It is deterministic in nature and will yeild the same result.

Maximize 
$$ Z = 3x_1 + 4x_2 $$

Subject to 
$$
\begin{align*}
2x_1 + x_2 &\leq 8 \\
x_1 + 2x_2 &\leq 6 \\
x_1, x_2 &\geq 0
\end{align*}
$$



### Integer Programming

- Branch and Bound method is used  where the problem is converted to LP and solved.
- The nearest floor and ceiling integer values bound the solution and prune the branches that are not satisfyying the constraints. 

### Non-convex Optimization Strategies


#### 1. Global Optimization Methods:
- **Branch and Bound**: Uses relaxations (often convex) to bound the function value and prune regions of the search space.
- **Simulated Annealing**: A probabilistic method that sometimes accepts worse solutions to escape local optima.
- **Genetic Algorithms**: Inspired by natural selection, this method employs mutation, crossover, and selection.
- **Particle Swarm Optimization**: Mimics the social behavior of birds and fishes, with a group of candidate solutions "flying" through the search space.

#### 2. Relaxation Techniques:
- Convert the problem to a convex one, solve it, and then derive solutions for the original problem.

#### 3. Sequential Convex Programming:
- Linearizes or convexifies the problem around a point and repeats the process iteratively. **TrajOpt** paper uses this.

#### 4. Restart Techniques:
- Methods like gradient descent can be started from various initial points to increase the chance of finding the global minimum.


